In [17]:
import pandas as pd
import os 
file_path = r"C:\Users\Mert Can\Desktop\tablodan_okuma.csv"
df = pd.read_csv(file_path, delimiter=";", decimal=",")
display(df)  # Print the DataFrame's contents


import math as mt
import sympy as sp
x = sp.symbols("x")
v_kmh= float(input("Rüzgar hızını girin (KM/H):"))
F1 = float(input(("Payload dan gelen kuvveti girin (N):")))
a1 = float(input("Bu girilen kuvvetinin masttan uzaklığını milimetre cinsinden girin (z-offset): " ))
b1 = float(input("Bu girilen kuvvetinin mastın kütle merkezinden kaçıklığını milimetre cinsinden girin (y-offset): "))
m1= float(input("Payload un kütlesini girin (kg):"))
a1 = a1/1000
b1 = b1/1000

print("Rüzgar hızı",v_kmh,"km/h değeri için:\n")
v = v_kmh/3.6
s1=1
s2=0
s3=0
s4=0
s5=0
s6=0
s7=0
s8=0
s9=0
s10=0
s11=0
s12=1
s13=0
s14=0
s15=1
L_list =[]
I_list =[]
F_list =[]

ML_list=[]
Ltop_list=[]
w_list=[]
FML_list=[]
FMLtoplam_list =[]
Moment_list=[]
y_list=[]
Rm_list=[]
sehim_list=[]
teta_list=[]

Ry = 0
E = 6.9*10**10 #Elastisite modülü in N/m2
n = len(df)

i=0
while s1 <=n:
    
    L = df.iloc[i,0]
    D = df.iloc[i,1]
    t = df.iloc[i,2]

    L = L/1000
    D = D/1000
    t = t/1000
    

    I = 1 / 64 * mt.pi * (D ** 4 - t ** 4)
    A =D*L
    F = 1 / 2 * 1.225 * v ** 2 * 1.2 * A   ##
    w = F/L
    
    w_list.append(w)
    L_list.append(L)
    I_list.append(I)
    F_list.append(F)
    s1+=1
    i+=1

    
Ry=sum(F_list) + F1                                                                                ###

print("boruların uzunlukları (m):",L_list)
print("F değerleri:",F_list )
print(f"Ry:{Ry} N")  
print("w değerleri:",w_list)
print("I değerleri listesi(m):",I_list)


       
while s2<n:
    if s2 ==0:
        ML=L_list[s2]/2
        ML_list.append(ML)
        s2+=1
    else:
        ML= ML_list[s3]+(L_list[s3]+L_list[s2])/2
        ML_list.append(ML)
        s2+=1
        s3+=1

#print("Yaratılan momentlerin bağlantı noktasına olan uzaklığı:",ML_list)  

while s9<n:
    a = F_list[s9]*ML_list[s9]               
    Rm_list.append(a)
    s9+=1


Ltop=0
j = 0
while s4 <n:
    Ltop = Ltop + L_list[j]
    Ltop_list.append(Ltop) 
    s4 += 1
    j += 1  # Increment j to access the next element

#print("Ltop_list:",Ltop_list) 

Rm = sum(Rm_list)  + F1*(Ltop_list[-1] + a1) +  m1*9.81*b1                                      ###
print("Rm:", Rm)

  

while s5 <n:
    FML=F_list[s5]*(x-ML_list[s5])
    FML_list.append(FML)
    s5+=1

FMLtoplam = 0

while s6<n:
    FMLtoplam = FMLtoplam + FML_list[s6]
    FMLtoplam_list.append(FMLtoplam)
    s6+=1


i=0
while s7<n:
    if s7==0:
        M = Ry*x - Rm - (w_list[s7]/2 * (x**2))
        Moment_list.append(M)
        s7+=1
    else:
        M = Ry*x - Rm - (FMLtoplam_list[i] + w_list[s7]/2 * (x-Ltop_list[i])**2)
        Moment_list.append(M)
        s7+=1
        i+=1

#print("Moment denklemleri:")
#for item in Moment_list:
#   print(item)
      

while s8 <n:
    f = Moment_list[s8]
    teta = sp.integrate(f, x)
    teta_list.append(teta)
    s8+=1

#print("teta değerleri:\n", teta_list)                              

while s11 < n-1:
    x_value=Ltop_list[s11]
    teta_value1 = teta_list[s11].subs(x,x_value)
    teta_value2 = teta_list[s12].subs(x,x_value)
    c = 1/I_list[s11]*(I_list[s12]*teta_value1 - I_list[s11]*teta_value2)
    teta_list[s12] = teta_list[s12] + c
    s11+=1
    s12+=1

#print("güncellenmiş teta değerleri:\n", teta_list)

while s13 < n:
    f = teta_list[s13]
    sehim = sp.integrate(f,x)
    y_list.append(sehim)
    s13+=1

while s14 < n-1:
    x_value = Ltop_list[s14]
    y_value1 = y_list[s14].subs(x,x_value)
    y_value2 = y_list[s15].subs(x,x_value)
    d = 1/I_list[s14]*(I_list[s15]*y_value1 - I_list[s14]*y_value2)
    y_list[s15] = y_list[s15] + d
    s14+=1
    s15+=1

x_value = Ltop_list[-1] 
ytop = y_list[-1].subs(x,x_value)/(E*I_list[-1]) * 1000
tetatop = teta_list[-1].subs(x,x_value)/(E*I_list[-1])
tetatop_gercek = tetatop*180/(mt.pi)

print("Maximum eğim:{} derece". format(tetatop_gercek))
print("Maximum sehim:{} mm".format(ytop))    

,Boru Uzunluğu (mm),Boru Çapı (mm),İç Çap (mm)
0,1713,125,116
1,1346,116,104
2,1346,104,96
3,1359,96,84
4,1333,84,72
5,1346,72,60
6,1339,60,48


Rüzgar hızı 80.0 km/h değeri için:

boruların uzunlukları (m): [1.713, 1.346, 1.346, 1.359, 1.333, 1.346, 1.339]
F değerleri: [77.71944444444445, 56.6715851851852, 50.80900740740741, 47.3536, 40.64168888888889, 35.175466666666665, 29.160444444444444]
Ry:487.53123703703716 N
w değerleri: [45.370370370370374, 42.103703703703715, 37.74814814814815, 34.84444444444444, 30.488888888888894, 26.13333333333333, 21.77777777777778]
I değerleri listesi(m): [3.0962697308620004e-06, 3.145425396627175e-06, 1.573309600917767e-06, 1.725299853498442e-06, 1.1247530018382188e-06, 6.829948092610352e-07, 3.755962512925812e-07]
Ltop_list: [1.713, 3.059, 4.405, 5.764, 7.097, 8.443000000000001, 9.782000000000002]
Rm: 2963.9080076629634
Maximum eğim:-5.45664380082247 derece
Maximum sehim:-512.787019636111 mm
